[![AWS SDK for pandas](_static/logo.png "AWS SDK for pandas")](https://github.com/aws/aws-sdk-pandas)

# 7 - Distributed Calls on Remote Cluster

[SDK for pandas](https://github.com/aws/aws-sdk-pandas) supports distribution of specific calls using [ray](https://www.ray.io/).

In [1]:

!pip install "awswrangler[distributed]==3.0.0a2"

## Configure and Build Ray Cluster in AWS.

#### Build Prerequisite Infrastructure

Build a security group and IAM instance profile for the Ray Cluster to use.

[<img src="https://s3.amazonaws.com/cloudformation-examples/cloudformation-launch-stack.png">](https://console.aws.amazon.com/cloudformation/home#/stacks/new?stackName=RayPrerequisiteInfra&templateURL=https://aws-data-wrangler-public-artifacts.s3.amazonaws.com/cloudformation/ray-prerequisite-infra.json)

#### Configure Ray Cluster Configuration
Start with a cluster configuration file (YAML).

In [ ]:
!touch config.yml

Replace all values to match your desired region, account number and name of resources deployed by the above CloudFormation Stack.

[Click here](https://console.aws.amazon.com/ec2/home?region=us-east-1#Images:visibility=public-images;search=:ray-amzn-wheels_latest_amzn_ray-1.9.2-cp38;v=3;$case=tags:false%5C,client:false;$regex=tags:false%5C,client:false) to find the Ray AMI for your desired region. The example configuration below uses the AMI for `us-east-1`

In [ ]:
cluster_name: pandas-sdk-dist-testing

initial_workers: 2
min_workers: 2
max_workers: 2

provider:
    type: aws
    region: us-east-1 # Change AWS region as necessary
    availability_zone: us-east-1a,us-east-1b,us-east-1c # Change as necessary
    security_group:
        GroupName: ray-cluster
    cache_stopped_nodes: False

available_node_types:
  ray.head.default:
    node_config:
      InstanceType: m4.xlarge
      IamInstanceProfile:
        # Replace with your account id and profile name if you did not use the default value
        Arn: arn:aws:iam::{ACCOUNT ID}:instance-profile/ray-cluster
      # Replace ImageId if using a different region / python version
      ImageId: ami-0ea510fcb67686b48

  ray.worker.default:
      min_workers: 2
      max_workers: 2
      node_config:
        InstanceType: m4.xlarge
        IamInstanceProfile:
          # Replace with your account id and profile name if you did not use the default value
          Arn: arn:aws:iam::{ACCOUNT ID}:instance-profile/ray-cluster
        # Replace ImageId if using a different region / python version
        ImageId: ami-0ea510fcb67686b48


setup_commands:
- pip install "awswrangler[distributed]==3.0.0a2"
- export AWS_DEFAULT_REGION=us-east-1

Provision Ray Cluster

In [ ]:
!ray up -y config.yml

Set `WR_ADDRESS` to Ray Cluster Address

In [ ]:
!export WR_ADDRESS="ray://$(ray get-head-ip config.yml | tail -1):10001"

You can now execute actions. The SDK will detect the required dependencies for its `distributed` mode and parallelize the methods that support it.

In [ ]:
import awswrangler as wr
print(f"Distributed Mode: {wr.config.distributed}")

Get Bucket Name

In [ ]:
import getpass 

bucket = getpass.getpass()

Read & write some data

In [ ]:
df = wr.s3.read_parquet(path="s3://ursa-labs-taxi-data/2010/02/data.parquet", parallelism=1000)
path="s3://{bucket}/parquet/taxi-data-02-2010.parquet"
wr.s3.to_parquet(df, path=path)

##### [More Info on Ray Clusters on AWS](https://docs.ray.io/en/master/cluster/vms/user-guides/launching-clusters/aws.html)